## Old data generation code

In [1]:
import pandas as pd
import numpy as np
import math
length = 510

In [2]:
annot_df = pd.read_csv('../../data/annotation/cannonical_annotation.csv',sep = '\t',header = 0,index_col = 0)
transcript_df = pd.read_csv('../../data/annotation/annot_label.csv',sep = '\t',header = 0)
transcript_df['length'] = np.absolute(transcript_df['Start']-transcript_df['End'])
#How many sequences if we fully chop transcriptome
np.sum(np.ceil(transcript_df['length']/length))
annot_df = annot_df[annot_df['Annotation'].isin(['exon','UTR',])]
annot_df['GeneID']=pd.Categorical(annot_df['GeneID'],transcript_df['Gene'])
annot_df = annot_df.sort_values(by=['Chrom','GeneID','Start']).reset_index()

In [3]:
reverse_strand = {'+':'-','-':'+'}

chrom = []
strand = []
start_l = []
end_l = []
gene = []
for index,row in transcript_df.iterrows():
    start = row['Start']
    end = row['End']
    split = [(round(length*i)+start, round(length*(i+1))+start) for i in range(int(math.ceil((end-start)/length)))]
    for entry in split:
        chrom.append(row['Chr'])
        strand.append(reverse_strand[row['Strand']])
        start_l.append(entry[0])
        end_l.append(entry[1])
        gene.append(row['Gene'])
    

In [4]:
bed_df = pd.DataFrame({'Chr':chrom,'Start':start_l,'End':end_l,'Strand':strand,'Gene':gene})
bed_df['Gene']=pd.Categorical(bed_df['Gene'],transcript_df['Gene'])
bed_df = bed_df.sort_values(by=['Chr','Gene','Start'])

In [5]:
bed_pointer = 0
annot_pointer = 0
label = np.empty(len(bed_df), dtype=object)
label[...] = [set()for _ in range(label.shape[0])]
#while bed_df.iloc[bed_pointer]['Gene'] == annot_df.iloc[annot_pointer]['GeneID']:
while bed_pointer < len(bed_df)-1  or annot_pointer < len(annot_df)-1:
    bed_row = bed_df.iloc[bed_pointer]
    annot_row = annot_df.iloc[annot_pointer]
    #chromosome check:
    if bed_row['Chr'] < annot_row['Chrom']:
        bed_pointer += 1
        continue
    elif bed_row['Chr'] > annot_row['Chrom']:
        annot_pointer += 1
        continue
    #not overlapping， move pointer 
    if bed_row['End'] <= annot_row['Start']:
        bed_pointer += 1
        continue
    elif bed_row['Start'] >= annot_row['End']:
        annot_pointer += 1
        continue
    #overlapping
    if max(bed_row['Start'], annot_row['Start']) <= min(bed_row['End'], annot_row['End']):
        #complete overlap:
        if (bed_row['Start'] >= annot_row['Start']) and (bed_row['End'] <= annot_row['End']):
            label[bed_pointer].add(annot_row['Annotation'])
            bed_pointer += 1 
        #partial overlap
        elif bed_row['Start'] <= annot_row['Start']:
            label[bed_pointer].update(['junction',annot_row['Annotation']])
            bed_pointer += 1
        #partial overlap
        elif bed_row['End'] >= annot_row['End']:
            label[bed_pointer].update([annot_row['Annotation'],'junction'])
            annot_pointer += 1
               

In [6]:
label_list = [list(ele) for ele in label]
label = [','.join(ele) for ele in label_list]
bed_df['Label'] = label
bed_df['Label'] = bed_df['Label'].replace(r'^\s*$', 'intron', regex=True)

In [8]:
bed_df.to_csv('../../data/pre-train/'+'test'+'/rna_selection.bed',sep = '\t', index=False, header = False)

In [8]:
from tqdm import tqdm

fasta = open('../data/pre-train/'+str(length)+'/rna_seq.fa', 'r')
lines = fasta.readlines()
seq = []
# Strips the newline character
for line in tqdm(lines):
    if line[0] == '>':
        next
    else: 
        seq.append(line.strip().upper())

100%|██████████| 7669382/7669382 [00:05<00:00, 1511546.34it/s]


In [9]:
import random
random.shuffle(seq)
data_length = len(seq)
train_data = seq[:int(data_length*0.8)]
valid_data = seq[int(data_length*0.8):int(data_length*0.9)]
test_data = seq[int(data_length*0.9):]

In [14]:
import h5py
import numpy as np
file_name = '../data/pre-train/'+str(length)+'/rna_seq.h5'
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('train',data = train_data)
h5f.create_dataset('valid',data = valid_data)
h5f.create_dataset('test',data = test_data)
h5f.close()

In [7]:
import h5py
import numpy as np
file_name = '../data/pre-train/rna_seq.h5'
h5f = h5py.File(file_name, 'r')['seq']

In [8]:
import re
pattern = '[^ACGTN]'
count = 0
for seq in h5f:
    seq = seq.decode('utf-8')
    search = re.search(pattern, seq, flags=0)
    if  search != None:
        count+=1
        print(search)
        break

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bed_df = pd.read_csv('../data/pre-train/region_selection.bed',sep='\t',index_col=None)

bed_df['Label'].unique()

Count of reads containing each label

'UTR':19953

'exon':363682

'junction': 284194

'intron': 3455744



Not balanced. Sub-sample?

Total read :3819756

## Annotation label double check

In [1]:
import pandas as pd
import numpy as np
import math
length = 510

In [2]:
annot_df = pd.read_csv('../../data/annotation/cannonical_annotation.csv',sep = '\t',header = 0,index_col = 0)
transcript_df = pd.read_csv('../../data/annotation/annot_label.csv',sep = '\t',header = 0)
transcript_df['length'] = np.absolute(transcript_df['Start']-transcript_df['End'])
annot_df = annot_df[annot_df['Annotation'].isin(['exon','UTR',])]
annot_df['GeneID']=pd.Categorical(annot_df['GeneID'],transcript_df['Gene'])
annot_df = annot_df.sort_values(by=['Chrom','GeneID','Start']).reset_index()

In [5]:
chrom = []
strand = []
start_l = []
end_l = []
gene = []
for index,row in transcript_df.iterrows():
    start = row['Start']
    end = row['End']
    split = [(round(length*i)+start, round(length*(i+1))+start) for i in range(int(math.ceil((end-start)/length)))]
    for entry in split:
        chrom.append(row['Chr'])
        strand.append(row['Strand'])
        start_l.append(entry[0])
        end_l.append(entry[1])
        gene.append(row['Gene'])
    

In [6]:
bed_df = pd.DataFrame({'Chr':chrom,'Start':start_l,'End':end_l,'Strand':strand,'Gene':gene})
bed_df['Gene']=pd.Categorical(bed_df['Gene'],transcript_df['Gene'])
bed_df = bed_df.sort_values(by=['Chr','Gene','Start'])

In [9]:
transcript_df.to_csv('/home/amber/multitask_RNA/data/pre-train/510/transcript.bed',
                index=False,header=False,sep='\t',columns=['Chr','Start','End','Gene','length','Strand'])

In [10]:
bed_df = bed_df[['Chr','Start','End','Gene','Gene','Strand']]
bed_df.to_csv('/home/amber/multitask_RNA/data/pre-train/510/rna_selection.bed',
                index=False,header=False,sep = '\t')

In [11]:
annot_df = annot_df[['Chrom','Start','End','Annotation','GeneID','Strand']]
annot_df.to_csv('/home/amber/multitask_RNA/data/pre-train/510/annot.bed',
                index=False,header=False,sep='\t')

In [ ]:
! bedtools subtract -s -a transcript.bed -b annot.bed > intron.bed

In [12]:
intron_df = pd.read_csv('../../data/pre-train/510/intron.bed',sep = '\t',header = None,
                        names = ['Chrom','Start','End','Annotation','GeneID','Strand'],index_col = None)
intron_df['GeneID'] = intron_df['Annotation']
intron_df['Annotation']='intron'
all_annot_df = pd.concat([intron_df,annot_df])

In [14]:
all_annot_df.to_csv('/home/amber/multitask_RNA/data/pre-train/510/annot.bed',
                index=False,header=False,sep='\t')

In [ ]:
! bedtools sort -i annot.bed > annot.sorted.bed

In [ ]:
! bedtools intersect -s -a ./rna_selection.bed -b ./annot.bed  -wa -wb > label.bed

In [15]:
final_df = pd.read_csv('../../data/pre-train/510/label.bed',sep = '\t',header = None,index_col = None)

In [38]:
bed_df.columns.values[4] = "Annotation"

In [22]:
unique_label = final_df.groupby([0,1,2,3,5])[9].unique()


In [41]:
test = unique_label.values
for i in range(len(test)):
    label_str = ','.join(test[i])
    test[i] = label_str

In [43]:
bed_df['Annotation'] = test

/tmp/ipykernel_205187/4113080471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bed_df['Annotation'] = test


In [45]:
bed_df.to_csv('/home/amber/multitask_RNA/data/pre-train/510/rna_selection.bed',
                index=False,header=False,sep='\t')

## Subsample sequence for balance class

In [25]:
import pandas as pd
import h5py
import numpy as np
from tqdm import tqdm
import random
length=510

In [22]:
bed_df = pd.read_csv('../../data/pre-train/510/rna_selection.bed',
                    sep = '\t',header = None,index_col = None)

In [23]:
labels=bed_df[4]
selected_id = []
intron_id = []
for i in range(len(labels)):
    label = labels[i].split(',')
    if (len(label) > 1) or ('UTR' in label) or ('exon' in label):
        selected_id.append(i)
    else:
        intron_id.append(i)
        

In [24]:
print(len(selected_id))
print(len(intron_id))
sub_size = int(0.5*len(selected_id))
selected_id.extend(random.choices(intron_id,k=sub_size))
print(len(selected_id))

480026
3354665
720039


In [35]:
fasta = open('../../data/pre-train/'+str(length)+'/rna_seq.fa', 'r')
line_number = np.array(selected_id)*2+1
lines = fasta.readlines()
seq = []
# Strips the newline character
for line_i in tqdm(line_number):
    if line[0] == '>':
        print('error in line count')
        break
    else:
        seq.append(lines[line_i].strip().upper())

100%|██████████| 720039/720039 [00:00<00:00, 822567.31it/s]


In [37]:
random.shuffle(seq)
data_length = len(seq)
train_data = seq[:int(data_length*0.9)]
valid_data = seq[int(data_length*0.9):]

In [38]:
file_name = '../../data/pre-train/'+str(length)+'/rna_seq.h5'
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('train',data = train_data)
h5f.create_dataset('valid',data = valid_data)
h5f.close()

FileNotFoundError: [Errno 2] Unable to create file (unable to open file: name = '../data/pre-train/510/rna_seq.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)